# <center> Анализ вакансий из HeadHunter. Работа с базой данных из Python.

In [1]:
# установим при необходимости адаптер базы данных PostgreSQL для Python
#!pip install psycopg2

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import psycopg2

In [3]:
# создаём соединение с базой данных
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## 1. Знакомство с данными

Ниже представлены таблицы, находящиеся в схеме ***public***

<img src = https://lms.skillfactory.ru/assets/courseware/v1/efd63819603e7d4f4433ed2fedec717c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/SQL_pj2_2_1.png alt="drawing" style="width:400px;">

Таблица ***VACANCIES*** хранит в себе данные по вакансиям и содержит следующие столбцы:

<img src = https://lms.skillfactory.ru/assets/courseware/v1/837cf6ff79f483e387a16c993634f3e4/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/SQL_pj2_2_2.png alt="drawing" style="width:600px;">

Таблица-справочник ***AREAS*** хранит код города и его название:

<img src = https://lms.skillfactory.ru/assets/courseware/v1/682c2306f3d46a25915a89d4ec7e16ed/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/SQL_pj2_2_3.png alt="drawing" style="width:600px;">

Таблица-справочник ***EMPLOYERS*** содержит список работодателей:

<img src = https://lms.skillfactory.ru/assets/courseware/v1/d2a26db623c75572c71923b57241e038/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/SQL_pj2_2_4.png alt="drawing" style="width:600px;">

Таблица-справочник ***INDUSTRIES*** содержит варианты сфер деятельности работодателей:

<img src = https://lms.skillfactory.ru/assets/courseware/v1/2c76bca09937a1a05a9e66d51008e298/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/SQL_pj2_2_5.png alt="drawing" style="width:600px;">

Таблица ***EMPLOYERS_INDUSTRIES*** необходима для организации связи между работодателями и сферами их деятельности.

*Эта таблица нужна нам, поскольку у одного работодателя может быть несколько сфер деятельности (или работодатели могут вовсе не указать их). Для удобства анализа необходимо хранить запись по каждой сфере каждого работодателя в отдельной строке таблицы.*

<img src = https://lms.skillfactory.ru/assets/courseware/v1/16ff3df0bb0ddecd922562f3c4bdd32c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/SQL_pj2_2_6.png alt="drawing" style="width:600px;">

## 2. Предварительный анализ данных

In [5]:
# Задание 3.1: определим количество вакансий в базе данных
query = f'''select 
               count (*) as vacancies_number
            from 
               public.vacancies
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies_number
0,49197


In [6]:
# Задание 3.2: определим количество работодателей в базе данных
query = f'''select 
               count (*) as employers_number
            from 
               public.employers
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employers_number
0,23501


In [7]:
# Задание 3.3: определим количество регионов в базе данных
query = f'''select
               count (id) as regions_number
            from 
               public.areas
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,regions_number
0,1362


In [8]:
# Задание 3.4: определим количество сфер деятельности в базе данных
query = f'''select 
               count (*) as industries_number
            from 
               public.industries
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industries_number
0,294


*Вывод*: в нашем распоряжении имеются базы данных, содержащие 49197 вакансий от 23501 работодателя. Представлены вакансии в 1362 городах и в 294 сферах деятельности. Количество городов в России составляет 1117 (на ноябрь 2022 г.). На основании этого можем сделать вывод, что в базе данных также содержатся вакансии из стран ближнего зарубежья.

## 3. Детальный анализ вакансий

In [9]:
# Задание 4.1: обозначим пятерку городов-лидеров по количеству вакансий
query = f'''select
               areas.name as area,
               count(vacancies.id) as cnt
            from
               public.vacancies
            join public.areas on vacancies.area_id = areas.id
            group by area
            order by cnt desc
            limit 5
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [10]:
# Задание 4.2: найдём количество вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой
query = f'''select
                count(vacancies.id) as cnt
            from
                public.vacancies
            where 
                salary_from is not null
            or
                salary_to is not null

         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt
0,24073


In [11]:
# Задание 4.3: найдём средние значения для нижней и верхней граниы зарплатной вилки
query = f'''select
                round(avg(salary_from)) as avg_salary_from,
                round(avg(salary_to)) as avg_salary_to
            from
                public.vacancies
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [12]:
# Задание 4.4: напишем запрос, который выведет количество вакансий для каждого сочетания типа рабочего 
# графика (schedule) и типа трудоустройства (employment), используемого в вакансиях
query = f'''select distinct
                schedule,
                employment,
                count(id) as cnt
            from
                public.vacancies s
            group by schedule, employment
            order by cnt desc
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


In [13]:
# Задание 4.5: напишем запрос, выводящий значения поля Требуемый опыт работы (experience) в 
# порядке возрастания количества вакансий, в которых указан данный вариант опыта
query = f'''select
                experience,
                count(id) as cnt
            from
                public.vacancies
            group by experience
            order by cnt asc
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


*Вывод:* 
* По количеству вакансий с большим отрывом по отношению к лругим городам лидирует город Москва. Что неудивительно, ведь Москва - самый крупный город по населению во всём СНГ.
* Ранее мы выяснили, что общее число вакансий в базе данных - 49197. И только в половине из них заполнено хотя бы одно из двух полей с указанием заработной платы. Согласно исследованию самого HeadHunter, большинство работодателей не указывают размер компенсации в вакансии, потому что готовы варьировать уровень предлагаемой зарплаты в зависимости от знаний, умений и других компетенций соискателя. А четверть компаний просто не хотят разглашать уровень своих зарплат.
* Средние значения зарплатной вилки в наших вакансиях: от 71000 до 110000 рублей. Реальная зарплатная вилка скорее всего будет выше, так как половина вакансий не содержит сведений об уровне заработной платы. Согласно исследованию HeadHunter, чем выше позиция соискателя, тем чаще кандидат узнает о размере заработной платы непосредственно на собеседовании.
* Подавляющее большинство вакансий (более 35000) подразумевают полную занятость и полный рабочий день. На втором месте по количеству вакансий (7800) - вакансии с полной занятостю и удалённой работой. В будущем количество вакансий "на удалёнке" будет только расти. С начала коронавирусных ограничений, только в России количество таких вакансий увеличилось втрое (по данным hh.ru).
* Наибольшее число вакансий (более 26000) для соискателей с опытом работы от 1 до 3 лет. Спрос на людей без опыта в 3.6 раза меньше. Возможно это связано с тем, что далеко не все компании готовы брать сотрудника без опыта и обучать, так как первое время на сотрудника без опыта работы необходимо тратить ресурсы более опытных сотрудников. На такие траты в большинстве случаев могут пойти только крупные компании.



## 4. Анализ работодателей

In [14]:
# Задание 5.1: выведем ТОП-5 работодателей по количеству вакансий
query = f'''select
                e.name,
                count(v.id) as cnt
            from
                public.vacancies v
            join public.employers e on v.employer_id = e.id
            group by e.name
            order by cnt desc
            limit 5
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [15]:
# Задание 5.2: напишем запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
# среди регионов, в которых нет вакансий, найдём тот, в котором наибольшее количество работодателей.
query = f'''select
                a.name,
                count(v.id) as vac_cnt,
                count(e.id) as empl_cnt
            from
                public.areas a
            left join public.vacancies v on v.area_id=a.id
            left join public.employers e on e.area = a.id
            group by a.name
            order by 
                vac_cnt asc,
                empl_cnt desc
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,vac_cnt,empl_cnt
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Ростовская область,0,18
...,...,...,...
1357,Новосибирск,1149438,1149438
1358,Алматы,1364132,1364132
1359,Минск,2354880,2354880
1360,Санкт-Петербург,6320667,6320667


In [16]:
# Задание 5.3: для каждого каждого работодателя посчитаем количество регионов, в которых он публикует свои вакансии
query = f'''select 
                e.name as employers,
                count(distinct v.area_id) as distinct_areas
            from
                public.vacancies v
            left join public.employers e on e.id = v.employer_id
            group by 1
            order by 2 desc
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employers,distinct_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


In [17]:
# Задание 5.4: найдём количество работодателей, у которых не указана сфера деятельности
query = f'''select
                count(e.id)
            from
                public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            where i.name is null
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,8419


In [12]:
# посмотрим на названия работодателей, у которых не указана сфера деятельности
query = f'''select distinct
                e.name
            from
                public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            where i.name is null
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name
0,1000 мелочей (ИП Гафиатуллина Гульшат Митарисо...
1,1001 Soft
2,100 печей
3,101 Плюс
4,1221Системс
...,...
8350,Ярославский завод упаковочных материалов
8351,Ярославский филиал ПГУПС
8352,Ярославский ЦГМС - филиал ФГБУ Центральное УГМС
8353,Ярпож Казань


In [18]:
# Задание 5.5.0: выведем ТОП-10 компаний с наибольшим количеством сфер деятельности
query = f'''select
                e.name as empl_name,
                count(i.name) as ind_name_cnt
            from
                public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            group by 1
            order by 2 desc
            limit 10
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,empl_name,ind_name_cnt
0,Модуль,16
1,Простые решения,11
2,Энергия,10
3,Прайд,10
4,Эксперт,9
5,Альфа,9
6,СБК,9
7,Прогресс,8
8,Автомастер,8
9,АТЛАНТ,8


In [19]:
 # Задание 5.5: напишем запрос, чтобы узнать название компании, находящейся на третьем месте 
# в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.
query = f'''select
                e.name as empl_name,
                count(i.name) as ind_name_cnt
            from
                public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            group by 1
            having count(i.name) = 4
            order by 1
            limit 3
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,empl_name,ind_name_cnt
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


In [20]:
# Задание 5.6.0: выведем ТОП-10 самых популярных сфер деятельности среди всех работодателей
query = f'''select
                i.name as ind_name,
                count(e.name) as empl_names_cnt
            from
                public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            group by 1
            having i.name is not null
            order by 2 desc
            limit 10
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ind_name,empl_names_cnt
0,Разработка программного обеспечения,3553
1,"Системная интеграция, автоматизации технологи...",2993
2,"Интернет-компания (поисковики, платежные систе...",1675
3,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798
4,Консалтинговые услуги,662
5,Кадровые агентства,389
6,"Электронно-вычислительная, оптическая, контрол...",374
7,Розничная сеть (продуктовая),368
8,Интернет-магазин,351
9,"Архитектура, проектирование",346


In [21]:
# Задание 5.6: выясним, у какого количества работодателей в качестве 
# сферы деятельности указана «Разработка программного обеспечения».
query = f'''select
                i.name as ind_name,
                count(e.name) as empl_names_cnt
            from
                public.employers e
            left join public.employers_industries ei on ei.employer_id = e.id
            left join public.industries i on i.id = ei.industry_id
            group by 1
            having i.name = 'Разработка программного обеспечения'
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ind_name,empl_names_cnt
0,Разработка программного обеспечения,3553


In [22]:
# Задание 5.7:
# Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены 
# вакансии компании, вместе с количеством вакансий в этих регионах. 
# Также добавьте строку Total с общим количеством вакансий компании.

# импортируем необходимые библиотеки для парсинга и для работы с регулярными выражениями
import requests
import re
from bs4 import BeautifulSoup

# необходимо получить список городов-миллионников со страницы сайта "Википедия"
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
sities_string = page.find('table', class_='standard').text
cities = re.findall('[А-Яа-я-\s]+', sities_string)
cities_list = tuple(cities[12:])

# создадим запрос для выполнения задания
query = f'''(select
                a.name area_name,
                count(v.name) vac_cnt
            from
                public.vacancies v
                join public.areas a on a.id = v.area_id
                join public.employers e on e.id = v.employer_id
            where 
                e.name = 'Яндекс' and a.name in {cities_list}
            group by 1
            order by 2)
            
            union all
            
            (select
                'Total',
                count(v.name)
            from
                public.vacancies v
                join public.areas a on a.id = v.area_id
                join public.employers e on e.id = v.employer_id
            where 
                e.name = 'Яндекс' and a.name in {cities_list})
            order by 2
            '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area_name,vac_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Ростов-на-Дону,25
5,Пермь,25
6,Казань,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


*Вывод:*
* Среди работодателей явно выделяется компания Яндекс. У Яндекса наибольшее общее количество вакансий (1933) и количество регионов, в которых публикуются вакансии (181).
* Больше всего вакансий публикуют крупные компании, такие как Тинькофф, Сбер, Газпром нефть и т.д.
* Почти треть работодателей (8419) не указали сферу деятельности. Как правило это небольшие компании или ИП.
* Компании, связанные с IT сферой, доминируют по количеству вакансий. Самая популярная сфера деятельности - разработка программного обеспечения.


## 5. Предметный анализ

In [23]:
# Задание 6.1: выясним, какое количество вакансий имеет отношение к данным (содержит в названии 'data' или 'данн')
query = f'''select
                count(id)
            from
                public.vacancies
            where lower(name) like '%data%' 
                or lower(name) like '%данн%'
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1771


In [24]:
# Задание 6.2: посмотрим, сколько вакансий есть для начинающего дата-сайентиста
query = f'''select
                count(id)
            from
                public.vacancies
            where (lower(name) like '%data scientist%'
                    or lower (name) like '%data science%'
                    or lower(name) like '%machine learn%'
                    or lower(name) like '%машинн%обучен%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%' and name not like '%HTML%')
                and (lower(name) like '%junior%'
                    or experience like 'Нет опыта'
                    or employment like 'Стажировка')
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,51


In [25]:
# Задание 6.3: выведем количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres
query = f'''select
                count(id)
            from
                public.vacancies
            where (lower(name) like '%data scientist%'
                    or lower (name) like '%data science%'
                    or lower(name) like '%machine learn%'
                    or lower(name) like '%машинн%обучен%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%' and name not like '%HTML%')
                and (lower(key_skills) like '%sql%' or lower(key_skills) like'%postgres%')
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,201


In [26]:
# Задание 6.4: выясним, насколько популярен Python в требованиях работодателей к DS. 
# Найдём количество вакансий, в которых в качестве ключевого навыка указан Python
query = f'''select
                count(id)
            from
                public.vacancies
            where (lower(name) like '%data scientist%'
                    or lower (name) like '%data science%'
                    or lower(name) like '%machine learn%'
                    or lower(name) like '%машинн%обучен%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%' and name not like '%HTML%')
                and key_skills like '%Python%'
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,351


In [27]:
# Задание 6.5: выведем средннее количество ключевых навыков в вакансиях для DS
query = f'''select
                round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2)
            from
                public.vacancies
            where (lower(name) like '%data scientist%'
                    or lower (name) like '%data science%'
                    or lower(name) like '%machine learn%'
                    or lower(name) like '%машинн%обучен%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%' and name not like '%HTML%')
                and key_skills is not null
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,round
0,6.41


In [88]:
# Задание 6.6: выведем средние зарплаты дата-сайентистов в зависимости от их опыта работы
query = f'''select
                experience,
                round(avg((coalesce(salary_to, salary_from) + coalesce(salary_from, salary_to))/2)) avg_salary
            from
                public.vacancies
            where (lower(name) like '%data scientist%'
                    or lower (name) like '%data science%'
                    or lower(name) like '%machine learn%'
                    or lower(name) like '%машинн%обучен%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%' and name not like '%HTML%')
                and (salary_from is not null or salary_to is not null)
            group by 1
         '''
df = pd.read_sql_query(query, connection)
df

c:\Users\alpis\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


*Вывод:*
* Среди всех вакансий 1771 имеем отношение к работе с данными. При этом для начинающего специалиста таких вакансий всего 51. Это снова подтверждает теорию о том, что далеко не все компании готовы тратить время и средства на обучение сотрудника без опыта. 
* Среднее количество ключевых навыков в вакансиях DS - 6.41.
* Полученные значения средних зарплат в зависимости от опыта работы очень близки к тем, что публикуются в различных профильных статьях в интернете. Не стоит забывать о том, что чем выше позиция соискателя, тем чаще кандидат узнает о размере заработной платы непосредственно на собеседовании. В таких вакансиях уровень заработной платы чаще всего не указывается. Решающую роль в уровне заработной платы специалиста DS имеет опыт работы и количество ключевых навыков.

### Финал

In [14]:
# и не забываем закрыть соединение после окончания работы
connection.close()